# imports

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt

In [3]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)

scanpy==1.4.5.post1 anndata==0.6.22.post1 umap==0.3.10 numpy==1.16.2 scipy==1.4.1 pandas==0.23.4 scikit-learn==0.22.2.post1 statsmodels==0.11.0 python-igraph==0.7.1 louvain==0.6.1


In [4]:
%matplotlib widget
# %matplotlib inline



# functions

In [5]:
def load_ST(ST_name,p,region,region_label,ST_root='/home/yzy/PUBDT/st/Spatial_transcriptomics/Spatial_maps/'):
    # read count
    # g个基因，n个细胞，每一行是一个基因，前两列是ENSEMBL／gene；其后是按顺序的细胞
    gene_list = []
    ENSEMBL_list = []
    cell_list = []
    count_list = []
    ST_count_file = ST_root+ST_name+'.count'

    ST_index_file = ST_root+ST_name+'.idx'
    spot_coor_dict = {}
    spot_list = []
    x_list = []
    y_list = []
    with open(ST_index_file) as f:
        i=0
        for line in f:
            line = line.strip()
            if i==0:

                spot_list_pre = line.split('\t')
                spot_list = spot_list_pre[1:]
            elif i==1:
                x_list_pre = line.split('\t')
                x_list = x_list_pre[1:]
                x_list = list(map(float,x_list))
            elif i==2:
                y_list_pre = line.split('\t')
                y_list = y_list_pre[1:]
                y_list = list(map(float,y_list))
            i+=1


    spots_list = []
    ENSEMBL_list = []
    gene_list = []
    count_list = []
    print('111')
    with open(ST_count_file,'r') as f:
        i=0
        for line in f:
            line = str.strip(line)
            line_sp = line.split('\t')
            if i==0:
                spots_list = line_sp[2:]
            else:
                ENSEMBL_list.append(line_sp[0])
                gene_list.append(line_sp[1])
                count_list.append(line_sp[2:])
            i+=1
        # g*n
    count_mat = np.array(count_list).astype('float32')



    x_list = np.array(x_list).astype('float32')
    y_list = np.array(y_list).astype('float32')

    in_X = np.transpose(count_mat)
    obs_name = spots_list
    # g = map(str,range(in_X.shape[1]))
    g = gene_list

    obs = pd.DataFrame(index=obs_name)

    # var_name must be str
    var = pd.DataFrame(index=g)
    # obs['barcode'] = np.array(index_list)
    var['ENSEMBL'] = np.array(ENSEMBL_list)
    # obs['true_labels'] = np.array(tenx_73k_label)
    # obs['true_labels'] = obs['true_labels'].astype('category')
    #     var['Genes'] = Genes
    adata = ad.AnnData(in_X,  obs=obs,var=var, dtype='float32')
    adata.obsm['spatial'] = np.concatenate((x_list.reshape(-1,1),y_list.reshape(-1,1)),axis=1)
    adata.obs['spatial_x'] = x_list
    adata.obs['spatial_y'] = y_list
    adata.obs['p'] = p
    adata.obs['p'] = adata.obs['p'].astype('category')
    adata.obs['region'] = region
    adata.obs['region'] = adata.obs['region'].astype('category')
    adata.obs['region_label'] = region_label
    return adata

# data loading

## ST

In [5]:
ST_root = '/home/yzy/PUBDT/st/Spatial_transcriptomics/Spatial_maps/'
ST_name = 'P3.2'

# read count
# g个基因，n个细胞，每一行是一个基因，前两列是ENSEMBL／gene；其后是按顺序的细胞
gene_list = []
ENSEMBL_list = []
cell_list = []
count_list = []
ST_count_file = ST_root+ST_name+'.count'

ST_index_file = ST_root+ST_name+'.idx'
spot_coor_dict = {}
spot_list = []
x_list = []
y_list = []
with open(ST_index_file) as f:
    i=0
    for line in f:
        line = line.strip()
        if i==0:
            
            spot_list_pre = line.split('\t')
            spot_list = spot_list_pre[1:]
        elif i==1:
            x_list_pre = line.split('\t')
            x_list = x_list_pre[1:]
            x_list = list(map(float,x_list))
        elif i==2:
            y_list_pre = line.split('\t')
            y_list = y_list_pre[1:]
            y_list = list(map(float,y_list))
        i+=1


spots_list = []
ENSEMBL_list = []
gene_list = []
count_list = []
print('111')
with open(ST_count_file,'r') as f:
    i=0
    for line in f:
        line = str.strip(line)
        line_sp = line.split('\t')
        if i==0:
            spots_list = line_sp[2:]
        else:
            ENSEMBL_list.append(line_sp[0])
            gene_list.append(line_sp[1])
            count_list.append(line_sp[2:])
        i+=1
# g*n
count_mat = np.array(count_list).astype('float32')



x_list = np.array(x_list).astype('float32')
y_list = np.array(y_list).astype('float32')
            

111


In [6]:
in_X = np.transpose(count_mat)
obs_name = spots_list
# g = map(str,range(in_X.shape[1]))
g = gene_list

obs = pd.DataFrame(index=obs_name)

# var_name must be str
var = pd.DataFrame(index=g)
# obs['barcode'] = np.array(index_list)
var['ENSEMBL'] = np.array(ENSEMBL_list)
# obs['true_labels'] = np.array(tenx_73k_label)
# obs['true_labels'] = obs['true_labels'].astype('category')
#     var['Genes'] = Genes
adata = ad.AnnData(in_X,  obs=obs,var=var, dtype='float32')
adata.obsm['spatial'] = np.concatenate((x_list.reshape(-1,1),y_list.reshape(-1,1)),axis=1)
adata.obs['spatial_x'] = x_list
adata.obs['spatial_y'] = y_list

# data integration

## data concat

In [6]:
p_list = np.arange(4)+1
# region_list = np.arange(3)+1
adata_list = []
region_label_dict = {
    (1,1):'NG',
    (1,2):'CG',
    (1,3):'CG',
    (2,1):'NG',
    (2,3):'IF',
    (2,4):'CG',
    (3,1):'IF',
    (3,2):'NG',
    (3,3):'CG',
    (4,1):'NG',
    (4,2):'NG',
    (4,3):'NG'
}
# e.g. p.region: P1.1
for p in p_list:
    if p==2:
        region_list = [1,3,4]
    else:
        region_list = [1,2,3]
    for region in region_list:
        cur_region_label = region_label_dict[p,region]
        cur_ST_name = 'P{0}.{1}'.format(p,region)
        cur_adata = load_ST(cur_ST_name,p,region,cur_region_label)
        adata_list.append(cur_adata)

111
111
111
111
111
111
111
111
111
111
111
111


## 检查剩下的基因

In [28]:
adata_list_check = [a.copy() for a in adata_list]

In [10]:
adata_concat_check = adata_list_check[0].concatenate(adata_list_check[1:])

In [11]:
adata_list_check_rg = [adata_concat_check[adata_concat_check.obs['batch']==str(i)] for i in range(12)]

In [47]:
sc.pp.scale(adata_list_check[1])

In [23]:
sc.pp.recipe_zheng17(adata_list_check[0])

running recipe zheng17
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
    finished (0:00:00)


In [48]:
sc.pp.pca(adata_list_check[1])
sc.pp.neighbors(adata_list_check[1],n_pcs=20,n_neighbors=20)
sc.tl.umap(adata_list_check[1])
sc.tl.leiden(adata_list_check[1])

computing PCA with n_comps = 50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 20
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:01)
running Leiden clustering
    finished: found 6 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:00)


In [49]:
sc.pl.umap(adata_list_check[1],color='leiden')

... storing 'region_label' as categorical


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
sc.pp.scale(adata_list_check_rg[1])
sc.pp.pca(adata_list_check_rg[1])
sc.pp.neighbors(adata_list_check_rg[1],n_pcs=20,n_neighbors=20)
sc.tl.umap(adata_list_check_rg[1])
sc.tl.leiden(adata_list_check_rg[1])

computing PCA with n_comps = 50


/home/yzy/anaconda3/envs/pytorch/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:909: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 20
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:01)
running Leiden clustering
    finished: found 6 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:00)


In [51]:
adata_list_check_rg[1].obs['leiden_ori'] = np.array(adata_list_check[1].obs['leiden'])

In [52]:
# fig,ax = plt.subplots(1,1,figsize=(10,5))
sc.pl.umap(adata_list_check_rg[1],color=['leiden','leiden_ori'])

... storing 'region_label' as categorical
... storing 'leiden_ori' as categorical


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
fig,axes = plt.subplots(1,2,figsize=(10,5))
sc.pl.scatter(adata_list_check[1],x='spatial_x',y='spatial_y',color='SPINK1',ax=axes[0])
sc.pl.scatter(adata_list_check_rg[1],x='spatial_x',y='spatial_y',color='leiden',ax=axes[1])

/home/yzy/.local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
sc.tl.rank_genes_groups(adata_list_check_rg[1],groupby='leiden',method='logreg')

ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
 (0:00:01)


/home/yzy/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [63]:
sc.pl.rank_genes_groups_heatmap(adata_list_check_rg[1],cmap='bwr',standard_scale='var',n_genes=5)

/home/yzy/.local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## bbknn

In [180]:
adata_list_bbknn = [a.copy() for a in adata_list]

In [181]:
adata_concat_bbknn = adata_list_bbknn[0].concatenate(adata_list_bbknn[1:])

In [182]:
sc.pp.recipe_zheng17(adata_concat_bbknn)

running recipe zheng17
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
    finished (0:00:00)


In [183]:
# before bbknn
sc.pp.pca(adata_concat_bbknn)
sc.pp.neighbors(adata_concat_bbknn,n_pcs=20,n_neighbors=20)
sc.tl.umap(adata_concat_bbknn)
sc.tl.louvain(adata_concat_bbknn)
sc.tl.leiden(adata_concat_bbknn)

computing PCA with n_comps = 50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 20
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:01)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:16)
running Louvain clustering
    using the "louvain" package of Traag (2017)
    finished: found 11 clusters and added
    'louvain', the cluster labels (adata.obs, categorical) (0:00:00)
running Leiden clustering
    finished: found 11 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:01)


In [184]:
sc.pl.umap(adata_concat_bbknn,color=['leiden','p','region_label','batch'])

... storing 'region_label' as categorical
/home/yzy/.local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [185]:
sc.external.pp.bbknn(adata_concat_bbknn, batch_key='batch')

computing batch balanced neighbors
	finished: added to `.uns['neighbors']`
	'distances', weighted adjacency matrix
	'connectivities', weighted adjacency matrix (0:00:01)


In [1]:
1+1

2

In [186]:
sc.tl.umap(adata_concat_bbknn)
sc.tl.leiden(adata_concat_bbknn)

computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:20)
running Leiden clustering
    finished: found 7 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:02)


In [198]:
sc.pl.umap(adata_concat_bbknn,color='batch',groups='11')

/home/yzy/.local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [187]:
sc.pl.umap(adata_concat_bbknn,color=['leiden','p','region_label','batch'])

/home/yzy/.local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## scanorama

In [130]:
adata_list_scanorama = [a.copy() for a in adata_list]

In [131]:
import scanorama

# # Integration.
# integrated = scanorama.integrate_scanpy(adata_list_scanorama)

# # Batch correction.
# corrected = scanorama.correct_scanpy(adata_list)

# Integration and batch correction.
integrated, corrected = scanorama.correct_scanpy(adata_list_scanorama, return_dimred=True)

Found 4121 genes among all datasets
[[0.         0.02216749 0.08796296 0.15904936 0.35196687 0.28472222
  0.07017544 0.13194444 0.00462963 0.62268519 0.02546296 0.02980132]
 [0.         0.         0.49261084 0.33743842 0.02955665 0.13300493
  0.04679803 0.         0.02463054 0.00985222 0.00308642 0.07389163]
 [0.         0.         0.         0.42230347 0.26501035 0.35044643
  0.02232855 0.36711281 0.32135729 0.02034884 0.05864198 0.21523179]
 [0.         0.         0.         0.         0.01449275 0.11151737
  0.14990859 0.01147228 0.02195609 0.03290676 0.0154321  0.03107861]
 [0.         0.         0.         0.         0.         0.35403727
  0.05175983 0.36438923 0.11387164 0.31677019 0.04012346 0.3126294 ]
 [0.         0.         0.         0.         0.         0.
  0.00223214 0.41071429 0.1497006  0.140625   0.00617284 0.22847682]
 [0.         0.         0.         0.         0.         0.
  0.         0.01338432 0.00399202 0.33173844 0.37345679 0.01324503]
 [0.         0.      

In [132]:
adata_concat_scanorama = corrected[0].concatenate(corrected[1:])

In [133]:
sc.pp.pca(adata_concat_scanorama)
sc.pp.neighbors(adata_concat_scanorama)
sc.tl.umap(adata_concat_scanorama)
sc.tl.leiden(adata_concat_scanorama)

computing PCA with n_comps = 50
    finished (0:00:02)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:14)
running Leiden clustering
    finished: found 18 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:00)


In [137]:
# adata_concat_scanorama.obs['region_label'] =adata_concat_bbknn.obs['region_label']

In [139]:
adata_concat_scanorama.obs['region_label'] =np.array(adata_concat_bbknn.obs['region_label'])

In [152]:
adata_concat_scanorama.obs['spatial_x'] = np.array(adata_concat_bbknn.obs['spatial_x'])
adata_concat_scanorama.obs['spatial_y'] = np.array(adata_concat_bbknn.obs['spatial_y'])

In [1]:
1+1

2

In [166]:
fig,axes = plt.subplots(4,3)
k = 0
for i in range(4):
    for j in range(3):
        sc.pl.scatter(adata_concat_scanorama[adata_concat_scanorama.obs['batch']==str(k)],
                      x='spatial_x',y='spatial_y',color='leiden',ax=axes[i][j],size=30,legend_loc='none')        
        k+=1
# sc.pl.scatter(adata_concat_scanorama[adata_concat_scanorama.obs['batch']=='2'],x='spatial_x',y='spatial_y',color='leiden')

/home/yzy/.local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [145]:
sc.pl.umap(adata_concat_scanorama,color='batch',groups='2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [141]:
sc.pl.umap(adata_concat_scanorama,color=['leiden','region_label','batch'])

... storing 'region_label' as categorical


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## mnn

## ingest

In [ ]:
## bbknn

# tmp

In [7]:
sc.pp.scale(adata)
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)
sc.tl.louvain(adata)

In [10]:
sc.tl.leiden(adata)

In [15]:
sc.pl.umap(adata,color=['louvain','leiden'])
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
sc.tl.paga(adata, groups='louvain')

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:00)


In [22]:
sc.pl.paga(adata, color=['louvain', 'leiden'])

--> added 'pos', the PAGA positions (adata.uns['paga'])


In [18]:
sc.tl.ingest

<function scanpy.tools._ingest.ingest(adata: anndata.core.anndata.AnnData, adata_ref: anndata.core.anndata.AnnData, obs: Union[str, Iterable[str], NoneType] = None, embedding_method: Union[str, Iterable[str]] = ['umap', 'pca'], labeling_method: str = 'knn', inplace: bool = True, **kwargs)>

In [19]:
import scanorama

/home/yzy/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [20]:
sc.tl.embedding_density(adata)
sc.pl.embedding_density(adata)

computing density on 'umap'
--> added
    'umap_density', densities (adata.obs)
    'umap_density_params', parameter (adata.uns)


In [24]:
sc.pl.embedding_density(adata)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …